❗ Оставлены рекомендации ревьюера


# Рекомендация тарифов

**Задача:**

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score


df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.head(20)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Данные по столбцам:

1. сalls — количество звонков,
2. minutes — суммарная длительность звонков в минутах,
3. messages — количество sms-сообщений,
4. mb_used — израсходованный интернет-трафик в Мб,
5. is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропусков в таблице нет. Звонки и сообщения переведу в целочисленный формат, количество звонков и сообщений дробным быть не может. Плюс мне кажется, что целочисленный формат обрабатывается быстрее (поправьте пж если это не так)  
Последний столбец делать булевым не будем, целочисленный формат подйдёт.

In [ ]:
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


In [ ]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Предобработка данных действительно не требуется, все данные выглядят адекватно


<div class="alert alert-info">
    
<b>Совет</b>
Также важно изучить корреляции в данных. Так мы можем оценить взаимосвязи в данных и влияние параметров на целевой признак. Рекомендую посмотреть метод `sns.pairplot` с параметром hue, в который мы передаем целевой признак. Будет здорово, если ты сможешь это реализовать в текущем или последующих проектах. </div>


## Разбейте данные на выборки

**Задача:**  
Разделить исходные данные на обучающую, валидационную и тестовую выборки.

У нас нет отдельной тестовой выборки, значит, данные нужно разбить на три части. Размеры тестового и валидационного наборов обычно равны, разобьём исходные данные в соотношении 3:1:1.  

Для этого мы разделим данные с помощью train_test_split() два раза. Сначала мы получим обучающую и тестовые выборки. Затем второй раз поделим обучающую снова и получим валидационную и окончательную обучающую выборки.

In [ ]:
df_train_first, df_valid = train_test_split(df, test_size=0.2, random_state=12345)

In [ ]:
df_train_first.shape

(2571, 5)

In [ ]:
df_valid.shape

(643, 5)

Всего у нас 3214 строки. Валидационная выборка — 20%, остальное приходится на обучающую.

In [ ]:
df_train, df_test = train_test_split(df_train_first, test_size=0.2, random_state=12345)

In [ ]:
df_train.shape

(2056, 5)

In [ ]:
df_test.shape

(515, 5)

Теперь на обучающую выборку приходится 60%, на тестовую и валидацонную по 20%.  

Дальше мы объявим переменные с признаками для обучения (target_train  и  features_train) и для проверки на валидационной выборке (target_valid и features_valid). Целевой признак — is_ultra, нам нужно, чтобы модель предлагала тариф исходя из поведения пользователей.

In [ ]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

## Исследуйте модели

### Решающее дерево

Сначала исследуем модель — решающее дерево. Чтобы определить наилучшую модель, будем изменять гиперпараметр max_depth. Попробуем поменять его от 1 до 20, перебрав эти значения в цикле

In [ ]:
for depth in range(1, 21):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split= 100)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7480559875583204
max_depth = 2 : 0.7838258164852255
max_depth = 3 : 0.7869362363919129
max_depth = 4 : 0.7931570762052877
max_depth = 5 : 0.7931570762052877
max_depth = 6 : 0.7931570762052877
max_depth = 7 : 0.7978227060653188
max_depth = 8 : 0.7993779160186625
max_depth = 9 : 0.7993779160186625
max_depth = 10 : 0.7993779160186625
max_depth = 11 : 0.7993779160186625
max_depth = 12 : 0.7978227060653188
max_depth = 13 : 0.7978227060653188
max_depth = 14 : 0.7978227060653188
max_depth = 15 : 0.7978227060653188
max_depth = 16 : 0.7978227060653188
max_depth = 17 : 0.7978227060653188
max_depth = 18 : 0.7978227060653188
max_depth = 19 : 0.7978227060653188
max_depth = 20 : 0.7978227060653188


Наилучший результат: max_depth = 9, дальше видно, что качество модели снижается: accuracy становится меньше. На всякий случай увеличим глубину до 100 и посмотрим, найдётся ли модель лучше.

In [ ]:

best_tree_model = None
best_tree_result = 0

for depth in range(1, 50, 7):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split=100)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    tree_result = accuracy_score(target_valid, predictions_valid)
    if tree_result > best_tree_result:
        best_tree_model = model_tree
        best_tree_result = tree_result

print("Accuracy наилучшей модели на валидационной выборке:", best_tree_result)

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625


Не нашлась — дальше уже качество модели снижается.

### Случайный лес

Теперь посмотрим на другой алгоритм классификации — случайный лес, возможно, он окажется более точным.  

Напишем цикл, который переберёт гиперпараметры: глубину и количество деревьев, и выдаст нам наилучшую accuracy.

In [ ]:
best_result = 0
best_depth = 0
best_est = 0

for depth in range(1, 16):
    for est in range(1, 100):
        forest_model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345, min_samples_split= 100)
        forest_model.fit(features_train, target_train)
        result = forest_model.score(features_valid, target_valid)
        if result > best_result:
            best_depth = depth # наилучшая глубина
            best_result = result
            best_est = est # наилучшее количество деревьев


print('Доля правильных ответов выборки для RandomForestClassifier:', best_result)
print('Оптимальная глубина деревьев:', best_depth)
print('Оптимальное количество деревьев:', best_est)

Доля правильных ответов выборки для RandomForestClassifier: 0.8055987558320373
Оптимальная глубина деревьев: 10
Оптимальное количество деревьев: 24


Результат лучше на 1%, чем у модели дерева принятия решений.

*P.S. Этот цикл у меня ооочень долго отрабатывает, около пары минут. Ок ли это при ручном подборе параметров или нужно урезать количество перебираемых значений?*

<div class="alert alert-block alert-success">

<b>Ответ:</b> Ручной подбор считается довольно быстрым методом, поэтому тут дело в перебираемых значений. Отличный способ это использовать шаг, `range(1, 100)` переберет **100 значений**, а `range(1, 100, 10)` с шагом переберет **10 значений** в том же диапазоне


### Логистическая регрессия

Перейдем к логистической регрессии:

Здесь будем перебирать только один гиперпараметр max_iter (максимальное количество итераций обучения).

In [ ]:
best_log_model = None
best_log_result = 0
best_iteration = 0

for iteration in range(100, 1001):
    log_model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iteration)
    log_model.fit(features_train, target_train)
    log_result = log_model.score(features_valid, target_valid)
    if log_result > best_log_result:
        best_log_model = log_model
        best_log_result = log_result
        best_iteration = iteration

print("Доля правильных ответов выборки для LogisticRegression:", best_log_result)
print('Оптимальное количество итераций:', best_iteration)

Доля правильных ответов выборки для LogisticRegression: 0.76049766718507
Оптимальное количество итераций: 100


Здесь результат получился хуже, чем и у случайного леса, и у дерева принятия решений, эту модель на тестовой выборке проверять точно не будем.

### Вывод

1. Обучили дерево принятия решений и выбрали лучшие с максимальной глубиной 9.
2. Обучили модель случайного леса, и выбрали лучшую с количеством деревьев 24 и глубиной 10. Эта модель оказалась самой точной (доля правильных ответов 80%), поэтому её мы будем проверять на тестовой выборке.
3. Обучили модель логистической регресии и получили долю правильных ответов на 3% хуже чем у дерева принятия решений и на 4% хуже, чем у модели случайного леса




<div class="alert alert-block alert-info">

<b>Совет:</b> Обрати также внимание на GridSearchCv и RandomizedSearchCV, эти инструменты могут быть удобными, когда большое количество параметров, уменьшает количество необходимых выборок, и делает код более лаконичным.
</div>

## Проверьте модель на тестовой выборке

Объявим переменные с признаками для тестовой выборки (target_test и features_test) и проверим модель на выборке, указав гиперпараметры: n_estimators=24, max_depth=10

In [ ]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [ ]:
# Создаём модель случайного леса
model = RandomForestClassifier(random_state=12345, n_estimators=24, max_depth=10)

# Обучаем
model.fit(features_train, target_train)

# Предсказываем целевой признак
predictions = model.predict(features_test)

# Считаем долю правильных ответов
accuracy = accuracy_score(target_test, predictions)

print('Доля правильных ответов тестовой выборки:', accuracy)

Доля правильных ответов тестовой выборки: 0.7864077669902912


### Вывод

На тестовой выборке модель случайного леса показала результат на 1% процент хуже, чем на валидационной. Тем не менее порог доли правильных ответов равный 0.75 она прошла.


<div class="alert alert-block alert-info">

<b>Совет:</b>  Перед тестированием можно объединять валидационную и тренировочную выборку в одну обучающую, это может дать нам немного качества
</div>


## (бонус) Проверьте модели на адекватность

Для проверки на адекватность мы сравним нашу модель с моделью, которая игнорирует входные данные (DummyClassifier).

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=12345)

dummy_clf.fit(features_train, target_train)

dummy_predictions = dummy_clf.predict(features_test)

accuracy = accuracy_score(target_test, dummy_predictions)

print('Доля правильных ответов при проверке на адекватность:', accuracy)

Доля правильных ответов при проверке на адекватность: 0.6757281553398058


### Вывод

Наша модель ошибается реже — в 22% случаев, в то время как модель, которая игнорирует входные данные — в 33% случаев. Думаю, можно сказать, что наша модель проверку на адекватность прошла.